In [77]:
import selectorlib
from selectorlib import Extractor
import requests
import json
from fake_useragent import UserAgent

# Create an Extractor by reading from the YAML file
e = Extractor.from_yaml_file('search.yml')

def scrape(url):
    ua = UserAgent()

    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': ua.random,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    # Download the page using requests
    #print("Downloading %s" % url)
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print(
                "Page %s was blocked by Amazon. Please try using better proxies\n" % url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d" % (
                url, r.status_code))
        return None
    # Pass the HTML of the page and create
    return e.extract(r.text)

#category = [ "headphones" ]
category = [ "processors", "laptops", "monitors", "mice", "keyboards", "cameras", "smartphones", "headphones" ]

for i in category:
    with open('search_' + str(i) + '_urls.txt', 'r') as urllist, open('search_' + str(i) + '_output.jsonl', 'w') as outfile:
        f = open(str(i) + '_urls.txt','w')
        for url in urllist.read().splitlines():
            data = scrape(url)
       
            if data:
                try:
                    for product in data['products']:
                        product['url'] = 'https://www.amazon.com' + product['url']
                        f.write(product['url'])
                        f.write('\n')

                        json.dump(product, outfile)
                        outfile.write(", \n")               
                except:
                    continue
        f.close()

In [76]:
for i in category:
    def scrape(url):
        ua = UserAgent()
        headers = {
            'dnt': '1',
            'upgrade-insecure-requests': '1',
            'user-agent': ua.random,
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/   signed-exchange;v=b3;q=0.9',
            'sec-fetch-site': 'same-origin',
            'sec-fetch-mode': 'navigate',
            'sec-fetch-user': '?1',
            'sec-fetch-dest': 'document',
            'referer': 'https://www.amazon.com/',
            'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        }
        r = requests.get(url, headers=headers)
        if r.status_code > 500:
            if "To discuss automated access to Amazon data please contact" in r.text:
                print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
            else:
                print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
            return None
        # Pass the HTML of the page and create 
        e = Extractor.from_yaml_file(str(i) + '.yml')
        return e.extract(r.text)
    # End of function
    with open(str(i) + '_urls.txt','r') as urllist, open(str(i) + '_output.jsonl','w') as outfile:
        for url in urllist.read().splitlines():
            data = scrape(url) 
            if data:
                try:
                    for xxx in [ "freq_bought_link", "link_to_all_reviews", "seller_link" ]:
                        data[xxx] = 'https://www.amazon.com' + data[xxx]
                    json.dump(data,outfile)
                    outfile.write("\n")

                except:
                    json.dump(data,outfile)
                    outfile.write("\n")